### Load data into Redis

*Redis structure*

| Name | Description |
|--------------------|
|flight:`<flight_id>`|hash set with price, availability, carrier|
|departure:|zset flights sorted by departure date|
|arrival:|zset flights sorted by arrival date|
|flights:`<origin>`:`<dest>`|set of flights for given route|
|routes:`<origin>`|set of destinations reachable from origin|
|distance:`<origin>`:`<dest>`|key value for route distance|
|carrier:`<carrier>`|set of flights by carrier|


In [2]:
import redis
import pandas as pd
import numpy as np
import datetime

ModuleNotFoundError: No module named 'redis'

In [ ]:
seconds_in_day = 24 * 60 * 60
seconds_in_hour = 60 * 60

In [ ]:
conn = redis.Redis(host='ec2-52-3-221-45.compute-1.amazonaws.com', port=6379)

flight_data = pd.read_csv('full_data.csv',
                          dtype={'flightID': np.int, 
                                 'Departure_epoch': np.int, 
                                 'Arrival_epoch': np.int,
                                 'RouteID' : np.int})

for _, row in flight_data.iterrows():
    flight = 'flight:' + str(row['flightID'])
    conn.hmset(flight, {
        'price' : row['Price'],
        'availability' : 100,
        'carrier' : row['Carrier']})
    conn.zadd('departure:', flight, int(row['Departure_epoch']))
    conn.zadd('arrival:', flight, int(row['Arrival_epoch']))
    conn.sadd('flights:' + row['Origin'] + ':' + row['Dest'], flight)
    conn.sadd('routes:' + row['Origin'], row['Dest'])
    conn.set('distance:' + row['Origin'] + ':' + row['Dest'], int(row['Distance']))
    conn.sadd('carrier:' + row['Carrier'], flight)
